## Предсказание будущих реакций на основе предшествующих

In [ ]:
%pylab inline
import matplotlib.animation as animation
import pandas as pd
import seaborn as sns
from IPython.display import HTML

In [ ]:
rc('animation', html='jshtml')
# если ошибка - обновите библиотеку matplotlib

In [ ]:
#загрузка данных
u='http://balatte.stireac.com/result.tsv/sherdim%40gmail.com/tuts__2143043410150713'
D = pd.read_csv(u, delimiter='\t')
D.info()

In [ ]:
sns.regplot('t','v',data=D); ylim(0);

Представим, что мы хотим предсказывать время реакции (ВР) непосредственно в ходе эксперимента. Другими словами мы хотим делать прогноз в режиме "реального времени".

Для упрощения представим, что мы ничего не знаем о заданном порядке МСИ и о модальности стимула.

В нулевой момент времени у нас нет данных. Мы прогнозируем, что ВР будет как у большинства людей от 150 до 350 мс. Возмем среднее значение 0.25 с.

In [ ]:
# заполним всю колонку этим значением
D['prognoz'] = 0.25
D.head()

Первые 9 секунд нашего эксперимента у нас все еще нет данных, и мы не можем оценить качество нашего прогноза. Как только появляются данные - мы можем рассчитать отклонение - среднеквадратичное.
(root-mean-square error (RMSE))

In [ ]:
sqrt(mean((D.v[0] - D.prognoz[0])**2))

В случае одного значения это все равно что абсолютное значение разности между прогнозируемым и реальным значениями.

Если значений много, то все равно будет одно число. Это число - метрика качества нашего прогноза, на основании которой мы можем сравнивать разные алгоритмы.

Есть несколько принципов прогноза:
1. Если что-то происходит сейчас так, то и в будущем будет так же.
2. Если есть четкая тенденция, то она продолжится.
3. Текущее значение может быть артефактом, т.е. возникло в результате случайного непредвиденного воздействия, которое больше может не повториться
4. Чем больше данных в прошлом, тем лучше можно сделать прогноз будущего

Мы попробуем сделать несколько логических шагов для понимания сути однонаправленного моделирования. Более зрелые модели доступны в специальных пакетах, на которые будут ссылки в конце.

In [ ]:
#сохраним текущий прогноз как первый вариант подхода "среднее по больнице"
D['pro_populatio']=D['prognoz']

#зададим функцию для удобства оценки качества прогноза
def quo(k):
    '''чем больше отклонение - тем ниже эффективность'''
    return 1/sqrt(mean((D.v - D[k])**2))
quo('pro_populatio')

Итого, если прогноз делать с потолка, то есть считать, что как у всех в среднем, то его эффективность 10 попугаев.

Возьмем на вооружение первый принцип. Будем ждать каждый раз повторения последнего значения.

In [ ]:
D.prognoz[1:]= D.v[:-1]
D.prognoz.fillna(method='ffill', inplace=True) #при пропусках оставляем предыдущий
D['pro_prev'] = D['prognoz']
quo('pro_prev')

Удивительно, но прогноз оказался хуже. Потому что постоянные 0.25 не так далеко от большинства значений в районе 0.3, а вот при прогнозе по предыдущему отдельные выбросы более 0.6 сделали свое черное дело: прогноз после них был крайне неудачным. 

In [ ]:
def play_prognoz(k='prognoz', n_future = 1, k0='v', interval=500):

    xx=D.index.values
    tt= xx[:-1]

    fig = figure()
    xlim(0,len(D.index)+1); ylim(0,0.7);

    hh=[]
    hh.append( plot([],[], color='b', lw=0, marker='d')[0] )
    hh.append( plot(xx[0],D[k][0], color='r', marker='*')[0] ) #prognoz
    hh.append( vlines([],[],[], colors='pink') ) #residuals

    def updatefig(i):
        hh[0].set_data(xx[:i+1], D[k0][:i+1])
        hh[1].set_data(xx[[i,i+n_future]], D[k][[i,i+n_future]])
        res1=vstack([tile(xx[i],(2,)), [D.loc[i,k0],D.loc[i,k]]]).T
        hh[2].set_segments((hh[2].get_segments() + [res1]))
                            
        return hh

    close()
    ani = animation.FuncAnimation(fig, updatefig, tt, interval=interval, blit=True, repeat=False)
    return ani #HTML(ani.to_html5_video())

play_prognoz('pro_prev')

Из-за выбросов становится понятно, что полагаться только на последнее значение опрометчиво. Лучше использовать для прогноза несколько последних значений.

Для ослабления влияния выбросов в качестве оценки центральной тенденции лучше использовать медиану.

In [ ]:
D['pro_median5']=D.v.rolling(5,  min_periods=1).median().shift()
play_prognoz('pro_median5')

Сравним эффективность предсказания по медиане с предсказанием по среднему.

In [ ]:
D['pro_mean5']=D.v.rolling(5,  min_periods=1).mean().shift()
quo('pro_median5'), quo('pro_mean5')

В данном случае среднее ведет себя лучше - 14 попугаев.

И среднее, и медиана являются способами оценить центральную тенденцию (*[M-estimator](https://en.wikipedia.org/wiki/M-estimator)*)


Вместо усреднения можно ввести взвешенное среднее, чтобы более экстремальные значения вносили меньший вклад, чем более типичные близкие к среднему. На практике используются разные [весовые функции]( http://statsmodels.sourceforge.net/stable/rlm_techn1.html).

Например, после первых 5 значений мы хотели бы иметь прогноз в районе 0.3, т.е. ослабить влияние 4-го значения, превышающего 0.4.

In [ ]:
x=D.v[:5]
z=((x - x.mean())/x.std()).values
z

wRamsay = exp(-0.3*abs(z))  #Ramsay’s Ea
wRamsay = wRamsay/sum(wRamsay)

t=1.345;  wHuber = [(t/_z if _z>t else 1.) for _z in abs(z)]
wHuber = wHuber/sum(wHuber)

mean(x), sum(x*0.2), sum(x*wRamsay), sum(x*wHuber)

Окна Рамсея и Хубера дали небольшое снижение относительно обычного среднего, в котором все слагаемые имеют одинаковый вес.

Мы берем 5 последних значений, значит средний возраст данных для прогноза - 3 временных шага. Получается, что если в серии происходят изменения - мы постоянно запаздываем в среднем на 3 шага назад.

Попробуем реализовать еще один принцип - *недавние события имеют большую предсказательную силу, чем предыдущие*. Для этого наделим большим весом последнее значение, а предыдущие - постепенно убывающими весами. 

In [ ]:
# weighted moving average (WMA)
w=(arange(5)+1)
w=w/w.sum()
bar(arange(5), w); xticks(arange(5), -(5-arange(5)))
title('Веса для последних 5 значений');

In [ ]:
rol = D.v.fillna(method='ffill').rolling(5)

D['pro_wma']=rol.apply(lambda x: sum(x*w[-len(x):])).shift()  #shifts to future on 1 step
quo('pro_wma')

In [ ]:
play_prognoz('pro_wma')

Популярное решение - *простое экспоненциальное сглаживание* (simple exponential smoothing (SES)) или exponentially weighted moving average (EWMA).

In [ ]:
# exponentially weighted moving average (EWMA)
alpha = 0.6
# (1 − α)2
w=alpha*((1-alpha)**arange(5,0,-1))
w=w/w.sum()
bar(arange(5), w); xticks(arange(5), -(5-arange(5)));

In [ ]:
rol = D.v.fillna(method='ffill').rolling(5)

D['pro_ewma']=rol.apply(lambda x: sum(x*w[-len(x):])).shift()
quo('pro_ewma')

In [ ]:
play_prognoz('pro_ewma')

Используя экпоненциально взвешенную среднюю мы получили качество в 10 попугаев. Нам все еще мешают выбросы. Попробуем объединить веса, ослабляющие экстремальные значения, с весами, ослабляющими старые значения. Это так называемая *конволюция*.

In [ ]:
def _conv1(x, alpha = 0.5):
    wEWMA=alpha*((1-alpha)**arange(5,0,-1))
    z=((x - x.mean())/x.std())
    w = exp(-0.3*abs(z))  #Ramsay’s Ea
    w=w*wEWMA
    w = w/sum(w)
    return sum(x*w)

_conv1(x)

In [ ]:
D['pro_conv1']=rol.apply(_conv1).shift()
quo('pro_conv1')

In [ ]:
def _conv2(x, alpha = 0.5, t=1.345):
    wEWMA=alpha*((1-alpha)**arange(5,0,-1))
    z=((x - x.mean())/x.std())
    w = [(t/_z if _z>t else 1.) for _z in abs(z)]
    w=w*wEWMA
    w = w/sum(w)
    return sum(x*w)

_conv2(x)

In [ ]:
D['pro_conv2']=rol.apply(_conv2).shift()
quo('pro_conv2')

In [ ]:
play_prognoz('pro_conv2')

Если мы уверены, что величина выброса не имеет значения, то можем загнать данные в заданные рамки. Это позволит более адекватно оценивать модели. Просто удалить выбросы при моделировании динамики нежелательно, поскольку мы нарушим ход времени. Замена на пустые значения также снизит информативность.

Например, в данном случае что 0.4, что 0.65 с характеризуют поздние реакции при пропуске стимула. Мы можем задать границу 0.4, чтобы все поздние реакции отклонялись в большую сторону, но при этом не так влияли на среднее значение. Такой прием, автоматически загоняющий выбросы в пределы заданных перцентилей, называется *винсоризацией*.

In [ ]:
import scipy.stats.mstats

D['vwin']=scipy.stats.mstats.winsorize(D.v, 0.1)
D.vwin.describe()

In [ ]:
D['prowin_conv2']=D.vwin.fillna(method='ffill').rolling(5).apply(_conv2).shift()
def quowin(k):
    return 1/sqrt(mean((D.vwin - D[k])**2))
quo('prowin_conv2')

In [ ]:
play_prognoz('prowin_conv2', k0='vwin')

Можно попробовать еще более продвинутые методы расчета весовых коэффициентов для подбора лучшего предсказания по группе последних значений, например [IRLS](https://en.wikipedia.org/wiki/Iteratively_reweighted_least_squares) с [функцией потерь Хубера](https://en.wikipedia.org/wiki/Huber_loss) или разные варианты ARIMA.

Кроме линейной можно попробовать сплайновую интерполяцию - `scipy.interpolate.spline`

### +++:

[Embedding Matplotlib Animations in Jupyter Notebooks](http://louistiao.me/posts/notebooks/embedding-matplotlib-animations-in-jupyter-as-interactive-javascript-widgets/)

[Moving average and exponential smoothing models](https://people.duke.edu/~rnau/411avg.htm)